In [ ]:
!pip install konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
Install mecab-python
/tmp /content
/content
Processing /tmp/mecab-python-0.996
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd
from konlpy.tag import Mecab
from collections import defaultdict

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 조리 과정별 파일 경로
cooking_processes = {
    '끓이기': '끓이기1000_2.csv',
    '굽기': '굽기1000_2.csv',
    '데치기': '데치기1000_2.csv',
    '무침': '무침1000_2.csv',
    '볶음': '볶음1000_2.csv',
    '부침': '부침1000_2.csv',
    '비빔': '비빔1000_2.csv',
    '삶기': '삶기1000_2.csv',
    '절임': '절임1000_2.csv',
    '조림': '조림1000_2.csv',
    '찜': '찜1000_2.csv',
    '튀김': '튀김1000_2.csv',
    '회': '회1000_2.csv'
}

# 조리 과정별 동사 리스트
cooking_verbs = {
    '끓이기': ['끓여', '끓', '끓이', '끓인', '끓어오르', '끓인다', '끓여서', '끓일', '끓여요', '끓였', '끓여야', '끓어오를', '끓어오른', '끓여도', '끓여진', '꿇', '들끓', '끓어오름', '끓여졌'],
    '굽기': ['굽', '굽히', '굽니', '구워', '구운', '구워진', '구우면', '구울', '구웠', '구', '구워서', '구워졌', '구우', '구워져', '달구', '구워낸', '구우실', '구워도', '구워낸다', '구할', '구우시면', '구우시는', '구워냈'],
    '데치기': ['데쳐', '데친', '데치', '데쳐서', '데칠', '데쳐진', '데쳐낸다', '데워', '데운', '데우', '데', '데워서', '데리', '데이', '데워준다', '데워진', '데운다'],
    '무침': ['무쳐', '무쳐도', '무쳐서', '무쳐야', '무쳤', '무치', '무친', '무친다', '무칠', '무침', '버무려', '버무려서', '버무려졌', '버무렸', '버무리', '버무린', '버무린다', '버무립니다', '버물', '버물려', '버물리', '버부'],
    '볶음': ['볶', '볶이'],
    '부침': ['부', '부쳐', '부르', '버부', '부쳐서', '부친', '부쳤'],
    '비빔': ['비비', '비벼서', '비빈', '비빌', '비빈다', '비벼지', '비벼준다', '비벼도', '비볐', '비빔', '비벼'],
    '삶기': ['삶', '삶기', '삶겨'],
    '절임': ['절여', '절인', '절여진', '절이', '절였', '절여졌', '절여서', '절인다', '절여질', '절', '절여져야'],
    '조림': ['졸여', '졸이', '졸아들', '졸', '졸인', '졸인다', '졸여진', '졸였', '졸일', '졸여야', '졸여서', '졸임', '졸려', '졸아드', '졸아든', '졸아붙', '졸린다', '졸리', '조려', '조리', '조린다', '조려서', '조', '조기'],
    '찜': ['찌', '찐다면', '찐', '찐다', '찔', '쪄서', '쪄낸', '쪄요', '쪄내', '쪄도', '쪄', '쪘'],
    '튀김': ['튀겨', '튀기', '튀긴', '튀길', '튀겨진', '튀', '튀긴다', '튀겼', '튀겨서', '튀겨야', '튀겨낸다', '튀겨져', '튀김한', '튀깁니다', '튀긴다고'],
    '회': ['썰', '썰린', '썰리', '썬']
}

# 각 조리 과정별 DF값
individual_df = defaultdict(int)
# 전체 조리 과정의 DF값
total_df = defaultdict(int)

# ...[이전 코드]...

# 모든 레시피를 포함하는 데이터프레임
all_recipes = pd.DataFrame()

# 각 조리 과정 파일을 읽어서 DF값 계산
for process, path in cooking_processes.items():
    try:
        data = pd.read_csv(f'/content/drive/My Drive/recipe/{path}', encoding='CP949')
        all_recipes = pd.concat([all_recipes, data], ignore_index=True)
        for verb in cooking_verbs[process]:
            individual_df[verb] = data['recipe'].str.contains(verb, na=False).sum()
    except UnicodeDecodeError:
        print(f"파일 {path}을(를) 읽을 수 없습니다. 인코딩을 확인하세요.")
        continue

# 전체 조리 과정의 동사 리스트 DF값 계산
for verb in set().union(*cooking_verbs.values()):
    total_df[verb] = all_recipes['recipe'].str.contains(verb, na=False).sum()

# 각 조리 과정의 레시피 수와 전체 레시피 수 계산
process_counts = defaultdict(int)
total_recipes = 0

for process, path in cooking_processes.items():
    process_counts[process] = all_recipes['recipe'].str.contains('|'.join(cooking_verbs[process]), na=False).sum()

total_recipes = len(all_recipes)

# 조건부 확률 계산
conditional_probabilities = defaultdict(dict)
for process, verbs in cooking_verbs.items():
    for verb in verbs:
        p_process = process_counts[process] / total_recipes
        p_verb_given_process = individual_df[verb] / process_counts[process]
        p_verb = total_df[verb] / total_recipes
        conditional_probabilities[process][verb] = (p_verb_given_process * p_process) / p_verb

# 결과 출력
print("1. 개별 조리 과정의 동사 리스트 DF값 계산:")
for verb, df in individual_df.items():
    print(f"'{verb}': {df}")

print("\n2. 전체 조리 과정의 동사 리스트 DF값 계산:")
for verb, df in total_df.items():
    print(f"'{verb}': {df}")

print("\n3. 조건부확률 계산:")
for process, verbs in conditional_probabilities.items():
    print(f"조리 과정 '{process}':")
    for verb, prob in verbs.items():
        print(f"P({process} | '{verb}') = {prob}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1. 개별 조리 과정의 동사 리스트 DF값 계산:
'끓여': 662
'끓': 852
'끓이': 292
'끓인': 178
'끓어오르': 120
'끓인다': 65
'끓여서': 59
'끓일': 52
'끓여요': 37
'끓였': 27
'끓여야': 13
'끓어오를': 10
'끓어오른': 8
'끓여도': 12
'끓여진': 4
'꿇': 2
'들끓': 1
'끓어오름': 1
'끓여졌': 1
'굽': 198
'굽히': 2
'굽니': 1
'구워': 742
'구운': 92
'구워진': 91
'구우면': 39
'구울': 32
'구웠': 28
'구': 844
'구워서': 20
'구워졌': 20
'구우': 61
'구워져': 16
'달구': 11
'구워낸': 13
'구우실': 9
'구워도': 13
'구워낸다': 3
'구할': 3
'구우시면': 2
'구우시는': 2
'구워냈': 2
'데쳐': 481
'데친': 291
'데치': 183
'데쳐서': 82
'데칠': 44
'데쳐진': 26
'데쳐낸다': 7
'데워': 6
'데운': 4
'데우': 3
'데': 656
'데워서': 2
'데리': 2
'데이': 3
'데워준다': 2
'데워진': 1
'데운다': 1
'무쳐': 407
'무쳐도': 10
'무쳐서': 18
'무쳐야': 5
'무쳤': 9
'무치': 70
'무친': 36
'무친다': 11
'무칠': 25
'무침': 337
'버무려': 299
'버무려서': 7
'버무려졌': 2
'버무렸': 2
'버무리': 55
'버무린': 52
'버무린다': 20
'버무립니다': 11
'버물': 15
'버물려': 2
'버물리': 1
'버부': 2
'볶': 860
'볶이': 41
'부': 404
'부쳐': 7
'부르': 4
'부쳐서': 2
'부친': 2
'부쳤': 1
'비비': 63
'